In [13]:
from time import sleep
from datetime import date, timedelta, datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

In [2]:
today = str(date.today())


In [3]:
now = str(datetime.now())
now

'2022-10-27 20:34:29.964463'

In [4]:
tomorrow = str(date.today() + timedelta(days=1))

In [4]:
"""
make it headless
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
  
# instance of Options class allows
# us to configure Headless Chrome
options = Options()
  
# this parameter tells Chrome that
# it should be run without UI (Headless)
options.headless = True
  
# initializing webdriver for Chrome with our options
driver = webdriver.Chrome(options=options)"""

'\nmake it headless\nfrom selenium import webdriver\nfrom selenium.webdriver.chrome.options import Options\n  \n# instance of Options class allows\n# us to configure Headless Chrome\noptions = Options()\n  \n# this parameter tells Chrome that\n# it should be run without UI (Headless)\noptions.headless = True\n  \n# initializing webdriver for Chrome with our options\ndriver = webdriver.Chrome(options=options)'

In [31]:
def degree_cleaner(to_clean):
    if "\n" in to_clean:
        to_clean = to_clean.split("\n")[0]
    if to_clean[len(to_clean)-1] == "°":
        return to_clean[:-1]
    else:
        return to_clean

In [44]:
def nws_cleaner(to_clean):
    return to_clean.split(' ')[1]

## Headers = city, prediction_date, time_now, service, temp

In [37]:
def ad_check(driver):
    try:
        ad = WebDriverWait(driver, 1).until(
            ec.visibility_of_element_located((By.XPATH, '//*[@id="ad_position_box"]')))
        print('ad')
    except:
        print('no ad')

In [74]:
to_write = []

driver = webdriver.Chrome(executable_path="/home/heath/bin/chromedriver")

driver.implicitly_wait(15)

# TWC NYC
driver.get("https://weather.com/weather/tenday/l/10035:4:US")
k = driver.find_elements(By.XPATH, "/html/body/div[1]/main/div[2]/main/div[1]/section/div[2]/details[2]/summary/div/div/div[1]/span[1]")

temp = degree_cleaner(k[0].text)

to_write.append(["NYC", tomorrow, now, "TWC", temp])

while True:
# Accuweather NYC
    try:
        driver.get("https://www.accuweather.com/en/us/central-park/10028/daily-weather-forecast/2627448?day=2")

        # rather than check ads, just retry it?
        WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[name()='svg' and @class='hamburger-button icon-hamburger']"))).click()
        WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[name()='svg' and @class='icon-settings']"))).click()
        drop = Select(driver.find_element(By.ID, "unit"))
        # I ran into an ad here
        drop.select_by_value("F")

        driver.back()

        temp_text = driver.find_element(By.XPATH, "/html/body/div/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]")
        temp = degree_cleaner(temp_text.text)
        
        break
    except:
        # if there's an ad, we just do it again
        pass

to_write.append(["NYC", tomorrow, now, "Accuweather", temp])

driver.get("https://forecast.weather.gov/MapClick.php?lat=40.7823&lon=-73.9654")

woop = driver.find_element(By.XPATH, '//*[@id="seven-day-forecast-list"]/li[2]/div/p[4]')

temp = nws_cleaner(woop.text)

to_write.append(["NYC", tomorrow, now, "NWS", temp])

driver.get("https://www.foreca.com/104887472/Chicago-Midway-International-Airport-Chicago-IL")
WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="drawerToggle"]'))).click()

WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, '//*[@class="title" and text()="F°"]'))).click()
driver.refresh()

temp = driver.find_element(By.XPATH, '//*[@id="pageContent"]/div/section[2]/div/div[1]/div[2]/a/p[2]/span[2]')
temp = degree_cleaner(temp.text)

to_write.append(["NYC", tomorrow, now, "Foreca", temp])

sleep(3)

#driver.close()

print(to_write)

/tmp/ipykernel_16283/2555017486.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/home/heath/bin/chromedriver")


[['NYC', '2022-10-28', '2022-10-27 20:34:29.964463', 'TWC', '56'], ['NYC', '2022-10-28', '2022-10-27 20:34:29.964463', 'Accuweather', '58'], ['NYC', '2022-10-28', '2022-10-27 20:34:29.964463', 'NWS', '57'], ['NYC', '2022-10-28', '2022-10-27 20:34:29.964463', 'Foreca', '61']]


In [75]:
driver.close()